In [44]:
%matplotlib inline
import pymysql
import pandas as pd 

# your username and password go here!
creds = {'user': 'fausto', 
         'pswd': 'dapuzzo'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [11]:
import pandas as pd
cl_epa = pd.read_csv("..\CategoryAndPageLinks\cl_epa_engineered.csv")
cl_nepa = pd.read_csv("..\CategoryAndPageLinks\cl_nepa_engineered.csv")

In [12]:
import numpy as np
cl_nepa.shape

(75837, 91)

In [13]:
col_for_db = cl_nepa.columns
col_for_db[:3]

Index(['page_id', '1961_births', '1967_births'], dtype='object')

In [ ]:
single = "{} INT(8) UNSIGNED NOT NULL"
createQ = list()
for i in range(len(col_for_db)):
    createQ.append(single.format(col_for_db[i]))    

In [ ]:
processing = ", ".join(createQ)

In [ ]:
processing[:100]

In [ ]:
tail = "PRIMARY KEY (page_id)) DEFAULT CHARACTER SET utf8"
CL_create = ", ".join([processing,tail])
header = "CREATE TABLE IF NOT EXISTS categorylinks ("
CL_create_f = " ".join([header, CL_create])

In [ ]:
cur.execute(CL_create_f)

In [16]:
import numpy as np
redundant = pd.merge(cl_epa, cl_nepa, how='inner', left_on="page_id", right_on="page_id")
redundantIDs = redundant['page_id'].tolist()
cl_nepa_filtered = cl_nepa.loc[~cl_nepa['page_id'].isin(redundantIDs)].copy()

In [17]:
print(len(redundantIDs))

73


In [18]:
master_links = pd.concat([cl_epa, cl_nepa_filtered])
master_links.shape

(83287, 91)

In [21]:
master_links["page_id"]

0              25
1             290
2             307
3             308
4             594
5             620
6             624
7             628
8             655
9             662
10            664
11            689
12            691
13            736
14            737
15            740
16            752
17            765
18            783
19            800
20            809
21            841
22            848
23            851
24            852
25            856
26            863
27            864
28            874
29            890
           ...   
75807    51090463
75808    51091252
75809    51091328
75810    51091550
75811    51092256
75812    51093744
75813    51093965
75814    51095029
75815    51095733
75816    51096098
75817    51096390
75818    51096414
75819    51097030
75820    51097120
75821    51097912
75822    51098561
75823    51099118
75824    51100454
75825    51101162
75826    51101460
75827    51102110
75828    51102592
75829    51103467
75830    51105617
75831    5

In [26]:
master_links = master_links.where((pd.notnull(master_links)), None) #converts NaNs to None type, required for mySQL

In [27]:
ma_for_db = master_links[col_for_db].to_dict(orient='records') #provides each row as a dict;
#keys are colnames, values are column values, the order of keys is retained from row to row

In [38]:
ma_for_db[42]

{'1961_births': 0,
 '1967_births': 0,
 '1971_births': 0,
 '1977_births': 0,
 '1992_births': 0,
 '20th_century_american_actresses': 0,
 '20th_century_american_male_actors': 0,
 '20th_century_american_singers': 0,
 '20th_century_american_writers': 0,
 '21st_century_american_actresses': 0,
 '21st_century_american_businesspeople': 0,
 '21st_century_american_male_actors': 0,
 '21st_century_american_singers': 0,
 '21st_century_american_writers': 0,
 '_alumni': 0,
 '_authors_list': 0,
 '_california': 0,
 '_contestants': 0,
 '_films': 0,
 '_members': 0,
 '_multiple_names': 0,
 '_winners': 0,
 'african_american_basketball_players': 0,
 'african_american_male_rappers': 0,
 'all_accuracy_disputes': 0,
 'all_npov_disputes': 0,
 'american_animated_films': 0,
 'american_child_actresses': 0,
 'american_children': 0,
 'american_christians': 0,
 'american_female_models': 0,
 'american_female_pop_singers': 0,
 'american_film_actresses': 0,
 'american_film_producers': 0,
 'american_hip_hop_singers': 0,
 

In [ ]:
cur.execute("SET autocommit=0") #makes batch inserts SO MUCH EASIER

In [ ]:
#generates the template query for insertions into db
insertQ = "INSERT IGNORE INTO categorylinks ({}) VALUES ({})"
cols = ", ".join(col_for_db)
val_format = ",".join(['%s'] * len(list(ma_for_db[0].keys())))
formatQ = insertQ.format(cols, val_format)
formatQ

In [ ]:
print(type(ma_for_db))

In [ ]:
for record in ma_for_db:
    values = [str(x) for x in list(record.values())]
    cur.execute(formatQ,values)

connection.commit()

In [49]:
cur.execute("SELECT page_id, united_states FROM categorylinks LIMIT 100")

2

In [50]:
rows = cur.fetchall() #returns tuple of tuples, preserving column order

In [51]:
rows

((0, 0), (1, 0))

In [52]:
easyPeasy = pd.DataFrame(list(rows), columns = ["page_id", "united_states"]) # columns = ["page_id", "pl_count", "pl_is_epa", "ratio"]
easyPeasy

,page_id,united_states
0,0,0
1,1,0


In [24]:
cur.close()
connection.close()